# Index using XGANDALF and Integration using rings

In [ ]:
from gandalf_iterator import gandalf_iterator

geomfile_path = "/home/bubl3932/files/LTA_sim/LTAsim.geom"
cellfile_path = "/home/bubl3932/files/LTA_sim/LTA.cell"
input_path =   "/home/bubl3932/files/LTA_sim/LTA-simulation"

output_file_base = "LTA"

num_threads = 24
x, y = 512.5, 512.5
step_size = 0.5
layers = 0
extra_flags=[
# PEAKFINDING
"--no-revalidate",
"--no-half-pixel-shift",
"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-biggest-pix=1",
"--min-sig=3",
"--local-bg-radius=10",
# INDEXING
"--indexing=xgandalf",
"--tolerance=10,10,10,5",
"--no-refine",
"--xgandalf-sampling-pitch=5",
"--xgandalf-grad-desc-iterations=1",
"--xgandalf-tolerance=0.02",
# "--xgandalf-no-deviation-from-provided-cell",
# INTEGRATION
"--integration=rings",
"--int-radius=4,5,9",
"--fix-profile-radius=70000000",
# OUTPUT
"--no-non-hits-in-stream",
]

"""Examples of extra flags(see crystfel documentation https://www.desy.de/~twhite/crystfel/manual-indexamajig.html):"""

""" Basic options
"--highres=n",
"--no-image-data",
"""

""" Peakfinding
"--peaks=cxi",
"--peak-radius=inner,middle,outer",
"--min-peaks=n",
"--median-filter=n",
"--filter-noise",
"--no-revalidate",
"--no-half-pixel-shift",

"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-peak-pix=6",
"--min-snr-biggest-pix=1",
"--min-sig=9",
"--min-peak-over-neighbour=5",
"--local-bg-radius=5",

"--peaks=peakfinder8",
"--threshold=45",
"--min-snr=3",
"--min-pix-count=3",
"--max-pix-count=500",
"--local-bg-radius=9",
"--min-res=30",
"--max-res=500",
"""

""" Indexing
"--indexing=xgandalf",

"--tolerance=tol"
"--no-check-cell",
"--no-check-peaks",
"--multi",
"--no-retry",
"--no-refine",

"--xgandalf-sampling-pitch=n"
"--xgandalf-grad-desc-iterations=n"
"--xgandalf-tolerance=n"
"--xgandalf-no-deviation-from-provided-cell"
"--xgandalf-max-lattice-vector-length=n"
"--xgandalf-min-lattice-vector-length=n"
"--xgandalf-max-peaks=n"
"--xgandalf-fast-execution"
"""

""" Integration
"--fix-profile-radius=n",
"--fix-divergence=n",
"--integration=rings",
"--int-radius=4,5,10",
"--push-res=n",
"--overpredict",
"--cell-parameters-only",
"""

""" Output
"--no-non-hits-in-stream",
"--no-peaks-in-stream",
"--no-refls-in-stream",
"--serial-offset
"""

gandalf_iterator(x, y, geomfile_path, cellfile_path, input_path, output_file_base, num_threads, step_size, layers, extra_flags=extra_flags)


'list.lst' has been created with 1 entries at /home/bubl3932/files/LTA_sim/LTA-simulation/list.lst


Processing XY pairs:   0%|          | 0/20 [00:00<?, ?it/s]

Running for x=-513.4, y=-512.7


This is what I understood your unit cell to be:
cubic P, right handed.
a      b      c            alpha   beta  gamma
 11.87  11.87  11.87 A     90.00  90.00  90.00 deg
List of indexing methods:
   0: xgandalf-nolatt-cell      (xgandalf using cell parameters as prior information)
Indexing parameters:
                  Check unit cell parameters: on
                        Check peak alignment: on
                   Refine indexing solutions: off
 Multi-lattice indexing ("delete and retry"): off
                              Retry indexing: on
60 images processed, 60 hits (100.0%), 48 indexable (80.0% of hits, 80.0% overall), 48 crystals, 12.0 images/sec.
114 images processed, 114 hits (100.0%), 74 indexable (64.9% of hits, 64.9% overall), 74 crystals, 10.5 images/sec.
168 images processed, 168 hits (100.0%), 90 indexable (53.6% of hits, 53.6% overall), 90 crystals, 10.7 images/sec.
213 images processed, 213 hits (100.0%), 109 indexable (51.2% of hits, 51.2% overall), 109 crystals, 9.0 

Running for x=-511.9, y=-513.1


63 images processed, 63 hits (100.0%), 57 indexable (90.5% of hits, 90.5% overall), 57 crystals, 12.6 images/sec.
131 images processed, 131 hits (100.0%), 110 indexable (84.0% of hits, 84.0% overall), 110 crystals, 13.1 images/sec.
186 images processed, 186 hits (100.0%), 150 indexable (80.6% of hits, 80.6% overall), 150 crystals, 11.0 images/sec.
240 images processed, 240 hits (100.0%), 179 indexable (74.6% of hits, 74.6% overall), 179 crystals, 10.8 images/sec.
295 images processed, 295 hits (100.0%), 219 indexable (74.2% of hits, 74.2% overall), 219 crystals, 10.8 images/sec.
349 images processed, 349 hits (100.0%), 252 indexable (72.2% of hits, 72.2% overall), 252 crystals, 10.6 images/sec.
393 images processed, 393 hits (100.0%), 275 indexable (70.0% of hits, 70.0% overall), 275 crystals, 8.7 images/sec.
440 images processed, 440 hits (100.0%), 292 indexable (66.4% of hits, 66.4% overall), 292 crystals, 9.2 images/sec.
488 images processed, 488 hits (100.0%), 320 indexable (65.6% 

In [ ]:

import os 
from run_partialator_and_convert import run_partialator_and_convert
from convert_hkl_crystfel_to_shelx import convert_hkl_crystfel_to_shelx 

stream_file = f"{input_path}/{output_file_base}_-{x}_-{y}.stream"
pointgroup = "m-3m"  # Point group of the crystal
num_threads = 24
iterations = 5

output_dir = run_partialator_and_convert(
    stream_file,
    pointgroup=pointgroup,
    num_threads=num_threads,
    iterations=iterations,
    convert_to_mtz=False,
    cell_file=cellfile_path
)

if output_dir is not None:
    print("All done. Results are in:", output_dir)


convert_hkl_crystfel_to_shelx(os.path.join(output_dir,"crystfel.hkl"))

In [ ]:
import os 
from convert_hkl_crystfel_to_shelx import convert_hkl_crystfel_to_shelx

convert_hkl_crystfel_to_shelx(os.path.join(output_dir,"crystfel.hkl"))